In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import imageio

In [84]:
letterDict={
  "a": 0,"b": 1,"c": 2,"d": 3,"e": 4,"f": 5,"g": 6,"h": 7,"i": 8,
    "j": 9,"k": 10,"l": 11,"m": 12,"n": 13,"o": 14,"p": 15,"q": 16,"r": 17,
    "s": 18,"t": 19,"u": 20,"v": 21,"w": 22,"x": 23,"y": 24,"z": 25
}

def readPictures(df_complete, directory,letter_value):
    im = imageio.imread(directory)
    im.shape
    im=np.reshape(im,400)
    df = pd.DataFrame(data=im).T
    y = np.repeat(0,400)
    df['Letter'] = letter_value
    
    return df_complete.append(df)


In [62]:

y = np.repeat(0,400)


In [ ]:
import os
all_imgs = pd.DataFrame()

def read_all_images(upper_directory,all_imgs):
    for subdir, dirs, files in os.walk(upper_directory):
        for file in files:
            #print os.path.join(subdir, file)
            filepath = subdir + os.sep + file
            if filepath.endswith(".jpg"):
                # print (filepath)
                x = filepath.split("/")
                letter = (x[-1][0])
                letter_val = letterDict.get(letter)

                all_imgs = readPictures(all_imgs,filepath,letter_val)
    return all_imgs

            
alldata = read_all_images("./dataset3",all_imgs)
alldata

In [58]:
MyEmptydf = pd.DataFrame()
nn1= pd.DataFrame([1,2,3]).T

MyEmptydf.append(nn1)

,0,1,2
0,1,2,3
